In [1]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = 'AUoIy5No3qjx3v3Q1S68U10qj'
consumer_secret = '16nMbFfRF4s5QpY7MqTVuEC0O6Zx45YUcOvalVXS0mXeBZZpGc'
access_token = '1013231814331305985-34iKKWvPXlHsaUsf7TK0ASwCOS9a9P'
access_secret = 'R86KqIjDaPdTPpnIw3ebmb8Lho0fySZRjrBSqHh3DpDa5'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [2]:
access_key = '1013231814331305985-34iKKWvPXlHsaUsf7TK0ASwCOS9a9P'

Extracted tweets from 2016 to 2018 from the folder 'GetOldTweets-python-master' <br>
Using the command ' python Exporter.py --near Austin --within 30mi --since 2016-01-10 --until 2018-09-10' in splits. <br>
Combine all these files into one single file <br>

# 1.1 Read and Stitch tweets

In [1]:
import numpy as np
import pandas as pd

In [2]:
del_cols = ['retweets','favorites','id','permalink']
cols = list(pd.read_csv("16_01-17_01.csv", nrows =1,sep=';'))
df1 = pd.read_csv("16_01-17_01.csv", error_bad_lines=False,sep=';',usecols =[i for i in cols if i not in del_cols])

In [3]:
del_cols = ['retweets','favorites','id','permalink']
cols = list(pd.read_csv("17_07-17_01.csv", nrows =1,sep=';'))
df2 = pd.read_csv("17_07-17_01.csv", error_bad_lines=False,sep=';',usecols =[i for i in cols if i not in del_cols])

In [4]:
del_cols = ['retweets','favorites','id','permalink']
cols = list(pd.read_csv("17_10-17_7.csv", nrows =1,sep=';'))
df3 = pd.read_csv("17_10-17_7.csv", error_bad_lines=False,sep=';',usecols =[i for i in cols if i not in del_cols])

In [5]:
del_cols = ['retweets','favorites','id','permalink']
cols = list(pd.read_csv("18_01-17_10.csv", nrows =1,sep=';'))
df4 = pd.read_csv("18_01-17_10.csv", error_bad_lines=False,sep=';',usecols =[i for i in cols if i not in del_cols])

In [6]:
del_cols = ['retweets','favorites','id','permalink']
cols = list(pd.read_csv("18_01-18_09.csv", nrows =1,sep=';'))
df5 = pd.read_csv("18_01-18_09.csv", error_bad_lines=False,sep=';',usecols =[i for i in cols if i not in del_cols])

In [7]:
df = pd.concat([df1,df2,df3,df4,df5],ignore_index=True)
del df1,df2,df3,df4,df5

In [8]:
###Tweets from  2016-01-10 to 2018-09-10 are copied to the df dataframe###
df.head()

,username,date,text,geo,mentions,hashtags
0,AustinTracyW,2017-01-08 17:59,# leftovers chicken fajita and grilled veggies...,NaN,NaN,#
1,adlongoria,2017-01-08 17:58,I'm at Violet Crown Cinema - @ vccinema for Ma...,NaN,@,NaN
2,supesec,2017-01-08 17:58,The Impalers # thisisaustinnotthatgreat # thei...,NaN,@Hotel,# #
3,sweetpeachyprop,2017-01-08 17:58,Fantastic dinner noms!!! (@Hyde Park Bar & Gri...,NaN,@Hyde @,NaN
4,JawnRedcorn,2017-01-08 17:56,he's all up in my George Foreman,NaN,NaN,NaN


In [9]:
###There are a total of 1,132,965 tweets for the two years
df.shape

(1132965, 6)

In [10]:
#save to one single file
df.to_csv('Total_tweets.csv')

# 2.1 Filtering

# 2.1.1 Web-scraping news articles to find crash related words

In [1]:
import requests
from bs4 import BeautifulSoup
###2018 fatalities###
list1 = []
urls = ["https://talk1370.radio.com/tag/2018-traffic-fatalities",\
      "https://talk1370.radio.com/tag/2018-traffic-fatalities?page=0%2C1",\
      "https://talk1370.radio.com/tag/2018-traffic-fatalities?page=0%2C2",\
      "https://talk1370.radio.com/tag/2018-traffic-fatalities?page=0%2C3"]

def getURL(page):
    """
    :param page: html of web page (here: Python home page) 
    :return: urls in that page 
    """
    start_link = page.find("a href")
    if start_link == -1:
        return None, 0
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote + 1)
    url = page[start_quote + 1: end_quote]
    return url, end_quote

for url in urls:
    response = requests.get(url)

    # parse html
    page = str(BeautifulSoup(response.content))

    while True:
        url, n = getURL(page)
        page = page[n:]
        if url:
            if url.startswith('/articles'):
                list1.append(url)
        else:
            break
list1 = [el for i, el in enumerate(list1) if el not in list1[:i]]

In [2]:
for i in range(0,len(list1)):
    list1[i] = 'https://talk1370.radio.com'+list1[i]

In [3]:
from newspaper import Article
list2 = []
for i in range(0,len(list1)-1):
    url = list1[i]
    article = Article(url)
    article.download()
    article.parse()
    list2.append(article.text)

In [4]:
urls = ["https://talk1370.radio.com/tag/2017-traffic-fatalities?page=0%2C1",\
      "https://talk1370.radio.com/tag/2017-traffic-fatalities?page=0%2C1",\
      "https://talk1370.radio.com/tag/2017-traffic-fatalities?page=0%2C2",\
      "https://talk1370.radio.com/tag/2017-traffic-fatalities?page=0%2C3",
      "https://talk1370.radio.com/tag/2017-traffic-fatalities?page=0%2C4",\
      "https://talk1370.radio.com/tag/2017-traffic-fatalities?page=0%2C5",\
      "https://talk1370.radio.com/tag/2017-traffic-fatalities?page=0%2C6"]
list1 = []
for url in urls:
    response = requests.get(url)

    # parse html
    page = str(BeautifulSoup(response.content))

    while True:
        url, n = getURL(page)
        page = page[n:]
        if url:
            if url.startswith('/articles'):
                list1.append(url)
        else:
            break
list1 = [el for i, el in enumerate(list1) if el not in list1[:i]]

In [5]:
for i in range(0,len(list1)):
    list1[i] = 'https://talk1370.radio.com'+list1[i]

In [6]:
for i in range(0,len(list1)):
    url = list1[i]
    article = Article(url)
    article.download()
    #article.parse()
    list2.append(article.text)

In [7]:
import re
for i in range(0,len(list2)):
    list2[i] = list2[i].replace("\\n","")
    list2[i] = re.sub("\s*-\s*", "", list2[i])
    list2[i] = re.sub('\W+',' ', list2[i] )
    list2[i] = re.sub(r'\b\w{1,2}\b', '', list2[i])

In [8]:
import pandas as pd
df1 = pd.DataFrame({'col':list2})

In [9]:
df1.to_csv('talk1370_articles.csv',index=False)

In [11]:
import pandas as pd
from collections import Counter 
df1 = pd.read_csv('talk1370_articles.csv')
#df1.drop(['Unnamed: 0'],axis=1,inplace=True)
list3 = Counter(" ".join(df1["col"]).split()).most_common(200)

In [12]:
list4 = []
for i in range(0,len(list3)):
    list4.append(list3[i][0])

In [13]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english')) 
list4 = [w for w in list4 if not w in stop_words]

In [14]:
df1 = pd.DataFrame({'col':list4})
df1.to_csv('frequent_words.csv',index=False)

In [1]:
import pandas as pd
df1 = pd.read_csv('frequent_words.csv')

# 2.1.2 Transportation and safety related words

# 2.1.3 Existing Annotated Datasets

# 3.1 Preprocessing

In [1]:
import pandas as pd
import nltk
df = pd.read_csv('Total_tweets.csv')

In [2]:
##remove urls##
df['text'] = df['text'].astype(str)
df['text'] = df['text'].str.replace('http\S+|www.\S+', '', case=False)

In [3]:
##remove special characters##
import re
df['text'] = df['text'].map(lambda x: re.sub(r'\W+', ' ', x))

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
df['text'] = [word_tokenize(i) for i in df['text']]
stop =stopwords.words('english')
df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stop])
df['text'] = df['text'].apply(lambda x: [item for item in x if len(item)>2]) #remove this and test again!

In [5]:
###crash related words among the frequent word list###
# list1 = ['Austin','crash','Police','scene','man','deceased','dead','morning','county','AustinTravis','Travis',\
#         'pronounced','accident','vehicle','traffic','block','injuries','struck','driver','male','fatal',\
#         'following','happened','investigation','reported','night','South','victim','transported','Medics',\
#         'Interstate','lanes','time','Southbound','died','road','northbound','APD','bus','boulevard','vehicles',\
#         'east','north','south','pedestrian','stop','Seton','Drive','truck','Hospital','Highway',\
#         'area','collision','incident','killed','injured','woman','afternoon','Lamar','traveling','southeast',\
#         'Street','arrived','hit','fatality']
list1 = ['crash','Police','scene','deceased','dead','pronounced','county','accident','vehicle','traffic','block',\
         'injuries','struck','driver','fatal','following','happened','investigation','reported','victim','transported',\
         'Medics','Interstate','lanes','time','Southbound','died','road','northbound','APD','boulevard','vehicles',\
        'pedestrian','stop','Drive','truck','Hospital','Highway',\
        'area','collision','incident','killed','injured','Lamar','traveling','southeast',\
        'Street','arrived','hit','fatality']

In [6]:
#get indices of tweets that don't have any words related to crash
list2 = []
a = 0
for i in df['text'].index:
    a = len(set(list1) & set(df['text'][i]))
    if a==0:
        list2.append(i)

In [7]:
df.shape

(1132965, 7)

In [9]:
df.drop(df.index[list2],inplace=True)

In [10]:
df.shape

(84480, 7)

In [11]:
df.to_csv('Filtered_tweets.csv',index=False)

In [12]:
random_subset = df.sample(n=1000)
random_subset.to_csv('random2.csv')

In [13]:
random_subset = df.sample(n=1000)
random_subset.to_csv('random3.csv')

In [14]:
random_subset = df.sample(n=1000)
random_subset.to_csv('random4.csv')

In [15]:
random_subset = df.sample(n=1000)
random_subset.to_csv('random5.csv')

In [30]:
# random_subset = df.sample(n=1000)
# random_subset.to_csv('random1.csv')

-- Some of the tweets have information about disabled vehicles. (With the hashtag ATXTraffic) <br>
-- Most of the accidents seem to be having hashtags (use this as a feature!!) <br>
-- Remove the word 'ATXtraffic' and check how many tweets are there by users <br>

# 3.2 Annotations for the dataset

In [1]:
import pandas as pd
r1 = pd.read_csv('random1.csv')
r1['Y/N'] = r1['Y/N'].astype(str)
r1 = r1[r1['Y/N'] == 'y']

In [11]:
r1.shape

(195, 8)

In [10]:
from collections import Counter 
Counter(" ".join(r1["text"]).split()).most_common()

[("'ATXtraffic']", 182),
 ("'Austin',", 135),
 ("['Accident',", 127),
 ("'traffic',", 83),
 ("'back',", 83),
 ("'delay',", 75),
 ("'mins',", 65),
 ("'stop',", 60),
 ("'accident',", 58),
 ("'Blvd',", 41),
 ("'reported',", 39),
 ("'near',", 37),
 ("'Travis',", 33),
 ("'cleared',", 31),
 ("['Serious',", 24),
 ("'Lamar',", 24),
 ("'183',", 20),
 ("'approaching',", 17),
 ("'290',", 15),
 ("'Ave',", 15)]

-- Out of the first random 1000 tweets, 195 are related to crashes, and 182 of them are from the ATXtraffic twitter account! <br>
-- Disabled vehicles, vehicles on fire, overturned etc. Accident - good news/bad news?

In [17]:
import pandas as pd
r2 = pd.read_csv('random2.csv')
r2['Y/N'] = r2['Y/N'].astype(str)
r2 = r2[r2['Y/N'] == 'y']

In [18]:
r2.shape

(250, 8)

In [20]:
Counter(" ".join(r2["text"]).split()).most_common()

[("'ATXtraffic']", 236),
 ("['Accident',", 158),
 ("'Austin',", 157),
 ("'traffic',", 107),
 ("'back',", 107),
 ("'delay',", 95),
 ("'accident',", 82),
 ("'mins',", 82),
 ("'stop',", 68),
 ("'Blvd',", 59),
 ("'reported',", 49),
 ("'Travis',", 49),
 ("'cleared',", 45),
 ("'near',", 38),
 ("'183',", 35),
 ("'Lamar',", 33),
 ("['Serious',", 30),
 ("'Mopac',", 28),
 ("['Major',", 27),
 ("'slow',", 24),
 ("'290',", 19),
 ("['Injury',", 19),
 ("'Ave',", 19),
 ("'Hwy',", 17),
 ("'approaching',", 16),
 ("'stopped',", 15),
 ("'blocked',", 13),
 ("'William',", 13),
 ("'Cannon',", 13),
 ("'Pkwy',", 12),
 ("'lane',", 12),
 ("'Airport',", 12),
 ("'Rundberg',", 11),
 ("'51st',", 11),
 ("'Oltorf',", 11),
 ("'Springs',", 10),
 ("'Cesar',", 10),
 ("'Chavez',", 10),
 ("'min',", 10),
 ("'Ben',", 10),
 ("'White',", 10),
 ("'left',", 9),
 ("'Parmer',", 9),
 ("'360',", 9),
 ("'Boulevard',", 8),
 ("'Braker',", 8),
 ("'Manor',", 8),
 ("'Service',", 8),
 ("'2222',", 7),
 ("'due',", 7),
 ("'130',", 7),
 ("'east

In [23]:
r3 = pd.read_csv('random3.csv')
r3['Y/N'] = r3['Y/N'].astype(str)
r3 = r3[r3['Y/N'] == 'y']

In [25]:
r3.shape

(259, 8)

In [24]:
Counter(" ".join(r3["text"]).split()).most_common()

[("'ATXtraffic']", 247),
 ("'Austin',", 173),
 ("['Accident',", 168),
 ("'traffic',", 107),
 ("'back',", 107),
 ("'delay',", 103),
 ("'mins',", 90),
 ("'accident',", 82),
 ("'stop',", 72),
 ("'Blvd',", 64),
 ("'reported',", 60),
 ("'near',", 43),
 ("'Travis',", 38),
 ("'183',", 34),
 ("'cleared',", 33),
 ("['Serious',", 31),
 ("'Lamar',", 31),
 ("'Mopac',", 31),
 ("['Injury',", 26),
 ("['Major',", 21),
 ("'Hwy',", 20),
 ("'Ave',", 20),
 ("'290',", 19),
 ("'slow',", 19),
 ("'stopped',", 18),
 ("'Pkwy',", 16),
 ("'blocked',", 15),
 ("'left',", 13),
 ("'360',", 12),
 ("'Ben',", 12),
 ("'White',", 12),
 ("'approaching',", 12),
 ("'Service',", 11),
 ("'lane',", 11),
 ("'Oltorf',", 10),
 ("'min',", 10),
 ("'130',", 9),
 ("'Riverside',", 9),
 ("'Parmer',", 9),
 ("'973',", 9),
 ("'Anderson',", 9),
 ("'lanes',", 9),
 ("'2222',", 9),
 ("'William',", 8),
 ("'Cannon',", 8),
 ("'Pflugerville',", 8),
 ("'Rundberg',", 8),
 ("'Manor',", 8),
 ("'Slaughter',", 8),
 ("'1st',", 8),
 ("'Braker',", 7),
 ("'

In [26]:
r4 = pd.read_csv('random4.csv')
r4['Y/N'] = r4['Y/N'].astype(str)
r4 = r4[r4['Y/N'] == 'y']

In [27]:
r4.shape

(269, 8)

In [28]:
Counter(" ".join(r4["text"]).split()).most_common()

[("'ATXtraffic']", 256),
 ("'Austin',", 204),
 ("['Accident',", 175),
 ("'traffic',", 117),
 ("'back',", 117),
 ("'delay',", 109),
 ("'accident',", 88),
 ("'mins',", 86),
 ("'stop',", 85),
 ("'Blvd',", 59),
 ("'reported',", 53),
 ("'near',", 51),
 ("'183',", 44),
 ("['Serious',", 39),
 ("'Travis',", 35),
 ("'Lamar',", 31),
 ("'cleared',", 30),
 ("['Injury',", 27),
 ("'Hwy',", 23),
 ("'approaching',", 22),
 ("'Ave',", 21),
 ("'blocked',", 20),
 ("'Mopac',", 20),
 ("'360',", 19),
 ("'lane',", 19),
 ("['Major',", 18),
 ("'slow',", 18),
 ("'Pkwy',", 16),
 ("'min',", 16),
 ("'right',", 14),
 ("'stopped',", 14),
 ("'Braker',", 14),
 ("'290',", 13),
 ("'Burnet',", 12),
 ("'2222',", 12),
 ("'Howard',", 11),
 ("'Ben',", 11),
 ("'White',", 11),
 ("'left',", 11),
 ("'Cesar',", 10),
 ("'Chavez',", 10),
 ("'51st',", 10),
 ("'Service',", 10),
 ("'Riverside',", 9),
 ("'Pflugerville',", 9),
 ("'William',", 9),
 ("'Cannon',", 9),
 ("'Boulevard',", 9),
 ("'Wells',", 8),
 ("'Branch',", 8),
 ("'Anderson',

In [ ]:
list1 = ['ATXtraffic','Accident','traffic','back','delay','accident','mins','stop','Blvd','reported',\
        'near','183','Serious','Travis','Lamar','cleared','Injury','Hwy','cleared','Major']

In [2]:
import pandas as pd
r2 = pd.read_csv('random2.csv',index_col=0)
r3 = pd.read_csv('random3.csv',index_col=0)
r4 = pd.read_csv('random4.csv',index_col=0)

In [3]:
df = pd.concat([r2,r3,r4])

In [4]:
df.shape

(3000, 7)

In [5]:
df = df[~df.index.duplicated(keep='first')]

In [6]:
df.shape

(2973, 7)

- FRAMEWORK

Features <br>
- Time of the day <br>
- User/organizaion <br>
- words(unigrams/bigrams) <br>
- hashtags <br>